In [1]:
libraries = c("dplyr","magrittr","tidyr","ggplot2","rstan","readxl")
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

require(zoo)
require(lubridate)

base_sz = 12 # base_size parameter
theme_set(theme_bw())

'%&%' = function(x,y) paste0(x,y)

options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

packageVersion("rstan")
packageVersion("StanHeaders")
rstan::stan_version()

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: lubridate


Attaching package: ‘lubridate’


The following object is masked from ‘package:base’:

    date




[1] ‘2.19.3’

[1] ‘2.21.0.1’

[1] "2.21.0"

In [3]:
# here we omit the extraction of the dataset from our master file, 
# but the resulting dataframe consists of six cases from in Li et al NEJM paper
Df %>% filter((SIStatus=='Certain')&(DiagnosisCountry=='China')) %>% select(ID, SIStatus, Onset, ExposureL, DateReportedConfirmed, InfectorID) %>% 
    mutate(ID = as.character(ID), 
           InfectorID = as.character(InfectorID), 
           SIStatus = as.character(SIStatus), 
           OnsetL = if_else(!is.na(Onset), Onset, ExposureL), 
           OnsetR = if_else(!is.na(Onset), Onset+days(1), DateReportedConfirmed)) %>%
    rename(S_L = OnsetL, S_R = OnsetR) -> df

lvls <- levels(Df$ID)
df

ID,SIStatus,Onset,ExposureL,DateReportedConfirmed,InfectorID,S_L,S_R
<chr>,<chr>,<dttm>,<dttm>,<dttm>,<chr>,<dttm>,<dttm>
NW667,Certain,2019-12-25,NA,NA,NW666,2019-12-25,2019-12-26
NW668,Certain,2019-12-29,NA,NA,NW666,2019-12-29,2019-12-30
NW670,Certain,2020-01-03,NA,NA,NW669,2020-01-03,2020-01-04
NW673,Certain,2019-12-19,NA,NA,NW671,2019-12-19,2019-12-20
NW677,Certain,2019-12-24,NA,NA,NW675,2019-12-24,2019-12-25
NW679,Certain,2020-01-11,NA,NA,NW678,2020-01-11,2020-01-12


In [4]:
Df %>% select(ID, Onset, ExposureL, DateReportedConfirmed) %>% 
    mutate(OnsetL = if_else(!is.na(Onset), Onset, ExposureL), 
           OnsetR = if_else(!is.na(Onset), Onset+days(1), DateReportedConfirmed)) %>%
    select(OnsetL, OnsetR, ID) %>% rename(E_L=OnsetL, E_R = OnsetR, InfectorID = ID) %>% 
    mutate(InfectorID = as.character(InfectorID)) %>%
    left_join(df %>% select(InfectorID, SIStatus, S_L, S_R), by='InfectorID') %>%
    drop_na %>% distinct %>% mutate_each(as.Date, -InfectorID, -SIStatus) %>% select(InfectorID, SIStatus, everything()) %>% arrange(SIStatus) -> df

df

InfectorID,SIStatus,E_L,E_R,S_L,S_R
<chr>,<chr>,<date>,<date>,<date>,<date>
NW666,Certain,2019-12-20,2019-12-21,2019-12-25,2019-12-26
NW666,Certain,2019-12-20,2019-12-21,2019-12-29,2019-12-30
NW669,Certain,2019-12-27,2019-12-28,2020-01-03,2020-01-04
NW671,Certain,2019-12-12,2019-12-13,2019-12-19,2019-12-20
NW675,Certain,2019-12-21,2019-12-22,2019-12-24,2019-12-25
NW678,Certain,2020-01-04,2020-01-05,2020-01-11,2020-01-12


In [5]:
CUTOFF_TIME = as.Date('2020-02-12')
t0 = as.Date('2019-12-01')

df['tstar'] = CUTOFF_TIME
df %<>% mutate_each(list(~(as.numeric(. - t0))), -InfectorID, -SIStatus) %>% mutate(dist = (S_R+S_L)/2-(E_R+E_L)/2)

df %<>% mutate(S_L = if_else(S_L < E_L, E_L, S_L), E_R = if_else(E_R > S_R, S_R, E_R)) 

df

InfectorID,SIStatus,E_L,E_R,S_L,S_R,tstar,dist
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NW666,Certain,19,20,24,25,73,5
NW666,Certain,19,20,28,29,73,9
NW669,Certain,26,27,33,34,73,7
NW671,Certain,11,12,18,19,73,7
NW675,Certain,20,21,23,24,73,3
NW678,Certain,34,35,41,42,73,7


In [6]:
mean(df$dist)

[1] 6.333333

In [7]:
sd(df$dist)

[1] 2.065591

In [8]:
data_drname = "../../data"
flname = 'data-probable.csv'
write.table(df, paste0(data_drname,flname), row.names=FALSE, sep=",", quote = FALSE)

# Stan simulations

In [9]:
stanmaindir = 'stan-sims-NEJMChina'
unlink(stanmaindir, recursive=T)
dir.create(stanmaindir)

# No truncation

## <font color="maroon">Lognormal distribution</font>

In [15]:
## main dir for Stan simulations
standirname = stanmaindir%&%"/lognormal-no_truncation"
unlink(standirname, recursive=T)
dir.create(standirname)

# Dumping data
N = nrow(df)
E_L = df$E_L
E_R = df$E_R
S_L = df$S_L
S_R = df$S_R
stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
e_raw = rep(.5, N)
s_raw = rep(.5, N)
logmean_SI = log(mean(df$dist))
logsd_SI = log(sd(df$dist))
stan_rdump(c('e_raw', 's_raw', 'logmean_SI', 'logsd_SI'), file=standirname%&%"/Init.R") 

# Stan program
"data {
    int<lower = 0> N; // number of records
    vector<lower = 0>[N] E_L;
    vector<lower = 0>[N] E_R;
    vector<lower = 0>[N] S_L;
    vector<lower = 0>[N] S_R;
}

parameters {
    real logmean_SI;
    real logsd_SI;

    vector<lower = 0, upper = 1>[N] s_raw;
    vector<lower = 0, upper = 1>[N] e_raw;
}

transformed parameters {
    real<lower = 0> param2 = sqrt(log((exp(2*(logsd_SI-logmean_SI))+1.0)));
    real<lower = 0> param1 = logmean_SI - param2^2/2.0;

    vector<lower = min(S_L), upper = max(S_R)>[N] s;
    vector<lower = min(E_L), upper = max(E_R)>[N] e;
    vector<lower = 0, upper = max(S_R)>[N] t;

    {
        vector[N] s_;

        
        s_ = S_L + (S_R - S_L) .* s_raw;
        for (k in 1:N) {
            if (s_[k] < E_L[k])
                s[k] = E_L[k];
            else 
                s[k] = s_[k];

            if (E_R[k] > s[k]) 
                e[k] = E_L[k] + (s[k] - 1e-3 - E_L[k]) * e_raw[k];
            else if (E_L[k] > s[k])
                e[k] = s[k] - 1e-3;
            else
                e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
        }
        t = s - e;
    }
}

model {
    logmean_SI ~ std_normal();
    logsd_SI ~ std_normal();

    e_raw ~ normal(0.5, 1.0);
    s_raw ~ normal(0.5, 1.0);

    t ~ lognormal(param1, param2);
}

generated quantities {
    real<lower = 0> mean_SI = exp(param1 + param2^2/2);
    real<lower = 0> sd_SI = sqrt((exp(param2^2)-1)*exp(2*param1+param2^2));

    vector[N] log_likelihood;
    for (k in 1:N) 
        log_likelihood[k] = lognormal_lpdf(t[k] | param1, param2);
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

standistribdir = "../../../CmdStan"
stanscriptdir = "../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/"%&%standirname
## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..10}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=10000 num_warmup=100000 save_warmup=0 \\
            adapt delta=0.98 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

## running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [3] "bin/stanc  --o=../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/stan-sims-NEJMChina/lognormal-no_truncation/fi

## <font color="maroon">Gamma distribution</font>

In [14]:
## main dir for Stan simulations
standirname = stanmaindir%&%"/gamma-no_truncation"
unlink(standirname, recursive=T)
dir.create(standirname)

# Dumping data
N = nrow(df)
E_L = df$E_L
E_R = df$E_R
S_L = df$S_L
S_R = df$S_R
stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
e_raw = rep(.2, N)
s_raw = rep(.8, N)
param1 = (mean(df$dist)/sd(df$dist))^2
param2 = mean(df$dist)/(sd(df$dist)^2)
stan_rdump(c('e_raw', 's_raw', 'param1', 'param2'), file=standirname%&%"/Init.R") 

# Stan program
"data {
    int<lower = 0> N; // number of records
    vector<lower = 0>[N] E_L;
    vector<lower = 0>[N] E_R;
    vector<lower = 0>[N] S_L;
    vector<lower = 0>[N] S_R;
}

parameters {
    real<lower=0> mean_SI;
    real<lower=0> sd_SI;

    vector<lower = 0, upper = 1>[N] e_raw;
    vector<lower = 0, upper = 1>[N] s_raw;
}

transformed parameters {
    real<lower = 0> param1 = (mean_SI/sd_SI)^2;
    real<lower = 0> param2 = mean_SI/(sd_SI^2);

    vector<lower = min(S_L), upper = max(S_R)>[N] s;
    vector<lower = min(E_L), upper = max(E_R)>[N] e;
    vector<lower = 0, upper = max(S_R)>[N] t;

    {
        vector[N] s_;
        
        s_ = S_L + (S_R - S_L) .* s_raw;
        for (k in 1:N) {
            if (s_[k] < E_L[k])
                s[k] = E_L[k];
            else 
                s[k] = s_[k];

            if (E_R[k] > s[k]) 
                e[k] = E_L[k] + (s[k] - 1e-3 - E_L[k]) * e_raw[k];
            else if (E_L[k] > s[k])
                e[k] = s[k] - 1e-3;
            else
                e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
        }
        t = s - e;
    }
}

model {
    mean_SI ~ normal(5.0, 10.0);
    sd_SI ~ cauchy(0, 5.0);

    e_raw ~ normal(0.5, 1.0);
    s_raw ~ normal(0.5, 1.0);

    t ~ gamma(param1, param2);
}

generated quantities {
    vector[N] log_likelihood;
    for (k in 1:N) 
        log_likelihood[k] = gamma_lpdf(t[k] | param1, param2);
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

standistribdir = "../../../CmdStan"
stanscriptdir = "../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/"%&%standirname
## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..10}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=10000 num_warmup=100000 save_warmup=0 \\
            adapt delta=0.98 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            refresh=1000 \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

## running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [3] "bin/stanc  --o=../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/stan-sims-NEJMChina/gamma-no_truncation/fit.hpp ../Hokkaido_Wu

## <font color="maroon">Weibull distribution</font>

In [13]:
## main dir for Stan simulations
standirname = stanmaindir%&%"/weibull-no_truncation"
unlink(standirname, recursive=T)
dir.create(standirname)

# Dumping data
N = nrow(df)
E_L = df$E_L
E_R = df$E_R
S_L = df$S_L
S_R = df$S_R
stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
e_raw = rep(.2, N)
s_raw = rep(.8, N)
logmean_SI = log(mean(df$dist))
param1 = 1.75
stan_rdump(c('e_raw', 's_raw', 'logmean_SI', 'param1'), file=standirname%&%"/Init.R") 

# Stan program
"data {
    int<lower = 0> N; // number of records
    vector<lower = 0>[N] E_L;
    vector<lower = 0>[N] E_R;
    vector<lower = 0>[N] S_L;
    vector<lower = 0>[N] S_R;
}

parameters {
    real<lower = 0> mean_SI;
    real<lower = 0> param1;

    vector<lower = 0, upper = 1>[N] e_raw;
    vector<lower = 0, upper = 1>[N] s_raw;
}

transformed parameters {
    real<lower = 0> param2 = mean_SI/tgamma(1.0+1.0/param1);

    vector<lower = min(S_L), upper = max(S_R)>[N] s;
    vector<lower = min(E_L), upper = max(E_R)>[N] e;
    vector<lower = 0, upper = max(S_R)>[N] t;

    {
        vector[N] s_;
        
        s_ = S_L + (S_R - S_L) .* s_raw;
        for (k in 1:N) {
            if (s_[k] < E_L[k])
                s[k] = E_L[k];
            else 
                s[k] = s_[k];

            if (E_R[k] > s[k]) 
                e[k] = E_L[k] + (s[k] - 1e-3 - E_L[k]) * e_raw[k];
            else if (E_L[k] > s[k])
                e[k] = s[k] - 1e-3;
            else
                e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
        }
        t = s - e;
    }
}

model {
    mean_SI ~ normal(5.0, 10.0);
    param1 ~ exponential(0.0001);

    e_raw ~ normal(0.5, 1.0);
    s_raw ~ normal(0.5, 1.0);

    t ~ weibull(param1, param2);
}

generated quantities {
    real sd_SI = param2*sqrt(tgamma(1.0+2.0/param1)-(tgamma(1.0+1.0/param1))^2);

    vector[N] log_likelihood;
    for (k in 1:N) 
        log_likelihood[k] = weibull_lpdf(t[k] | param1, param2);
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

standistribdir = "../../../CmdStan"
stanscriptdir = "../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/"%&%standirname
## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..10}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=10000 num_warmup=100000 save_warmup=0 \\
            adapt delta=0.98 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

## running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
 [3] "bin/stanc  --o=../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/stan-sims-NEJMChina/weibull-no_truncation/fit.hpp ../H